In [13]:
%pip install git+https://github.com/cair/tmu.git
%pip install numpy==1.26.4

  Cloning https://github.com/cair/tmu.git to /private/var/folders/3f/ltvwyfm53nb435_nt2n_7b9h0000gn/T/pip-req-build-sp8ij1xw
  Running command git clone --filter=blob:none --quiet https://github.com/cair/tmu.git /private/var/folders/3f/ltvwyfm53nb435_nt2n_7b9h0000gn/T/pip-req-build-sp8ij1xw
  Resolved https://github.com/cair/tmu.git to commit fd3d39f7cb0b28393c2d8463a6d93b6d363d487f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.3 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pickle

from tmu.models.autoencoder.autoencoder import TMAutoEncoder

2025-02-24 18:39:08,834 - tmu.util.cuda_profiler - WARNING - Could not import pycuda: No module named 'pycuda'
2025-02-24 18:39:08,835 - tmu.clause_bank.clause_bank_cuda - ERROR - No module named 'pycuda'
Traceback (most recent call last):
  File "/Users/kjellhaaland/Documents/GitHub/uia-master-thesis/.venv/lib/python3.12/site-packages/tmu/clause_bank/clause_bank_cuda.py", line 41, in <module>
    from pycuda._driver import Device, Context
ModuleNotFoundError: No module named 'pycuda'
2025-02-24 18:39:08,839 - tmu.clause_bank.clause_bank_cuda - WARNING - Could not import pycuda. This indicates that it is not installed! A possible fix is to run 'pip install pycuda'. Fallback to CPU ClauseBanks.


In [6]:
def load_train_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)

    # Take only the first 2000 rows
    X = X[:2000]

    return X

In [7]:
def log_results(epoch, precision, recall):
    with open("results.txt", "a") as f:
        f.write(f"{epoch},{precision},{recall}\n")

In [8]:
def save_model(tm: TMAutoEncoder, filename: str):
    a, d = tm.X_train, tm.encoded_X_train

    tm.X_train = None
    tm.encoded_X_train = None

    with open(filename, "wb") as f:
        pickle.dump(tm, f)

    tm.X_train = a
    tm.encoded_X_train = d

In [9]:
# TODO: Load X_train from a dataset
X_train = load_train_dataset("B", 34)

In [11]:
def train(args):
    tm = TMAutoEncoder(
        number_of_clauses=args["num_clauses"],
        T=args["T"],
        s=args["s"],
        output_active=args["output_active"],
        max_included_literals=args["max_included_literals"],
        accumulation=args["accumulation"],
        feature_negation=args["feature_negation"],
        platform=args["platform"],
        output_balancing=args["output_balancing"],
    )

    print(f"Starting training for {args['epochs']} epochs")

    for e in range(args["epochs"]):
        tm.fit(X_train, number_of_examples=args["number_of_examples"])

        save_model(tm, "latest_new.pkl")
        print(f"Epoch: {e + 1}")


In [12]:
number_of_features = 63 * 5
output_active = np.arange(number_of_features, dtype=np.uint32)

number_of_clauses = 1024

args: dict = {
    "clause_weight_threshold": 0,
    "number_of_examples": 10000,
    "output_active": output_active,
    "accumulation": 1,
    "num_clauses": number_of_clauses,
    "T": int(number_of_clauses * 0.75),
    "s": 10,
    "epochs": 50,
    "platform": "CPU",
    "output_balancing": 0,
    "max_included_literals": number_of_features * 2,
    "feature_negation": True,
}

result = train(args)

Starting training for 50 epochs
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9


KeyboardInterrupt: 